Notebook correspondant à la section 4.1.2 du rapport, pour la résolution des équations sur $M_x$ et $M_y$ couplées de la précession amortie :
\begin{equation}
\left\{
    \begin{aligned}
        \frac{dM_x}{d\tilde{t}} &= - M_y\omega_z - \lambda M_x M_z \omega_z \\
        \frac{dM_y}{d\tilde{t}} &= M_x\omega_z - \lambda M_y M_z \omega_z
    \end{aligned}
\right.
\end{equation}

In [ ]:
#imports
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

Physical parameters

In [ ]:
#initial value for M = [Mx, My] vector
M_0 = tf.constant([1,0], dtype='float32') 
#initial value for Mz
M_z0 = tf.constant(0, dtype='float32') 
W = 2*np.pi
lamb = -0.3

Definition of the ODE

In [ ]:
def ODE(Mx, My, Mz, W, lamb) :
    dMx = -W*(My + lamb*tf.multiply(Mx,Mz_tensor))
    dMy = W*(Mx - lamb*tf.multiply(My,Mz_tensor))
    return dMx, dMy

Training parameters

In [ ]:
N = 40 #number of samples for the independant variable
t_a = -1
t_b = 1
training_points = np.linspace(t_a,t_b,N)
training_points = tf.convert_to_tensor(training_points, dtype=tf.float32)

load_model = False
load_filename = "models/4_1_2_Mx_My"
save_model = False
save_filename = "models/4_1_2_Mx_My"


Load the model for the evolution of Mz

In [ ]:
Mz_model_filename = "models/4_1_1_Mz"
Mz_model = tf.keras.models.load_model(Mz_model_filename)
Mz_NN = Mz_model(training_points)

Mz_tensor = M_z0+training_points*Mz_NN[:,0]

Initializing the network

In [ ]:
# Network Parameters
n_input = 1     # input layer number of neurons
n_hidden_1 = 16 # 1st layer number of neurons
n_output = 2    # output layer number of neurons

#model definition :
model = tf.keras.Sequential([
  tf.keras.layers.Dense(n_hidden_1, activation=tf.nn.sigmoid, input_shape=(n_input,)),  
  tf.keras.layers.Dense(n_output)
])

if load_model :
    model = tf.keras.models.load_model(load_filename)



Loss function
https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/GradientTape


In [ ]:
def loss_function(model, input_tensor, M_0, lamb, Mz_tensor):

    with tf.GradientTape(persistent=True) as tape:
        tape.watch(input_tensor)
        output = model(input_tensor)
        Mx, My = M_0[0]+input_tensor*output[:,0], M_0[1]+input_tensor*output[:,1]

    dMx = tape.gradient(Mx, input_tensor)
    dMy = tape.gradient(My, input_tensor)

    dMx_target, dMy_target = ODE(Mx, My, Mz_tensor, W, lamb)

    ex = dMx - dMx_target
    ey = dMy - dMy_target

    return tf.reduce_mean(ex**2 + ey**2)

Gradient of loss

In [ ]:
def grad(model, input_tensor, M_0, lamb, Mz_tensor):
    with tf.GradientTape() as tape:
        loss_value = loss_function(model, input_tensor, M_0, lamb, Mz_tensor)

    gradient = tape.gradient(loss_value, model.trainable_variables)

    return loss_value, gradient

Training the neural network

In [ ]:
learning_rate = 3e-3
epochs = 2000
display_step = min(max(1,epochs//100), 1000)
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)

In [ ]:
losses = []
epochs_displayed = []

for epoch in range(epochs) :
    loss_value, grads = grad(model, training_points, M_0, lamb, Mz_tensor)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    if epoch % display_step == 0 :
        print("Loss after",epoch,"/",epochs,"epochs :",loss_value.numpy())
        losses.append(loss_value.numpy())
        epochs_displayed.append(epoch)

loss_value, grads = grad(model, training_points, M_0, lamb, Mz_tensor)
print("Final loss after",epochs,"epochs :",loss_value.numpy())
losses.append(loss_value.numpy())
epochs_displayed.append(epochs)

In [ ]:
if save_model :
    model.save(save_filename)

Plot the evolution of loss

In [ ]:
plt.plot(epochs_displayed, losses)
plt.yscale('log')
plt.show()


Compute the analytic solution

In [ ]:
plotting_points = np.linspace(t_a,t_b,200)
plotting_points = tf.convert_to_tensor(plotting_points, dtype=tf.float32)


#for plotting_points
K = (M_z0-1) / (M_z0+1)
exp_vector = tf.exp(-lamb*W*plotting_points)
F = exp_vector*(K-1)/(K*exp_vector**2-1)

Mx_ana =tf.multiply( F,(  tf.cos(W*plotting_points)*M_0[0] - tf.sin(W*plotting_points)*M_0[1]))
My_ana = tf.multiply(F,( tf.sin(W*plotting_points)*M_0[0] + tf.cos(W*plotting_points)*M_0[1]))

#for training_points
K = (M_z0-1) / (M_z0+1)
exp_vector = tf.exp(-lamb*W*training_points)
F = exp_vector*(K-1)/(K*exp_vector**2-1)

Mx_training =tf.multiply( F,(  tf.cos(W*training_points)*M_0[0] - tf.sin(W*training_points)*M_0[1]))
My_training = tf.multiply(F,( tf.sin(W*training_points)*M_0[0] + tf.cos(W*training_points)*M_0[1]))



Plot the estimation and the analytic solution

In [ ]:
#neural network estimation
output = model(plotting_points)
Mx_NN, My_NN = M_0[0]+plotting_points*output[:,0], M_0[1]+plotting_points*output[:,1]


#3D plotting
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.plot(        xs=Mx_NN,
                ys=My_NN,
                zs=plotting_points,
                label='solution approchée')

ax.plot(        xs=Mx_ana,
                ys=My_ana,
                zs=plotting_points,
                label='solution exacte')



ax.scatter(     xs=Mx_training,
                ys=My_training,
                zs=training_points,
                color='red',
                label='training points')

plt.legend()
plt.title('résolution par réseau de neurones')
plt.show()

plt.plot(plotting_points, Mx_NN, label='solution approchée')
plt.plot(plotting_points, Mx_ana, label='solution analytique')
plt.scatter(training_points, Mx_training,color='red', label="points d'entraînement")
plt.ylim(-1.1, 1.1)
plt.legend()
plt.show()
plt.plot(plotting_points, My_NN, label='solution approchée')
plt.plot(plotting_points, My_ana, label='solution analytique')
plt.scatter(training_points, My_training,color='red', label="points d'entraînement")
plt.legend()
plt.show()

We can check if the norm of the magnetization found is conserved :

In [ ]:
#neural network estimation
output = model(training_points)
Mx_NN, My_NN = M_0[0]+training_points*output[:,0], M_0[1]+training_points*output[:,1]
norm = tf.sqrt(Mx_NN**2 + My_NN**2 + Mz_tensor**2)

plt.plot(training_points, norm, label='Norm of M over time')
plt.scatter(training_points, norm, label='Norm of M over time')
plt.legend()
plt.show()